In [26]:
import tensorflow as tf
import numpy as np

In [27]:
# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

* one hot encoding

  * 데이터가 가질 수 있는 값들을 일렬로 나열한 배열을 만들고, 그 중 표현하려는 값을 뜻하는 인덱스의 원소만 1로 표기하고 나머지 원소는 모두 0으로 채우는 표기법
  
기타 = [1, 0, 0]  
포유류 = [0, 1, 0]  
조류 = [0, 0, 1]

In [28]:
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [29]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [30]:
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [31]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

In [32]:
model = tf.nn.softmax(L)   #softmax : 결과값의 전체 합이 1이 되도록 만들어주는 함수

In [33]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))

reduce_xxx 함수들은 텐서의 차원을 줄여준다. 함수 이름의 xx 부분이 구체적인 차원 축소 방법을 뜻하고, axis parameter로 축소할 차원을 정한다.  
예를들어 reduce_sum(input tensor, axis=1)은 주어진 텐서의 1번째 차원(???)의 값들을 다 더해(값 1개로 만들어서) 그 차원을 없앤다는 뜻

In [37]:
# 기본적인 경사하강법으로 최적화한다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화한다.
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행한다.
    for step in range(100):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})

        # 학습 도중 10번에 한 번씩 손실값을 출력해보자
        if (step + 1) % 10 == 0:
            print(step+1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    prediction = tf.argmax(model, axis=1)
    target = tf.argmax(Y, axis=1)
    print('예측값: ', sess.run(prediction, feed_dict={X: x_data}))
    print('실제값: ', sess.run(target, feed_dict={Y: y_data}))
    
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


10 1.1228555
20 1.1190923
30 1.1154586
40 1.1119488
50 1.1085578
60 1.1052803
70 1.1021115
80 1.0990465
90 1.096081
100 1.0932106
예측값:  [0 1 1 0 0 1]
실제값:  [0 1 2 0 0 2]
정확도: 66.67


정확도가 그렇게 높지 않은데 이는 신경망이 한 층밖에 되지 않기때문이다. 층을 하나더 늘려주면 해결할 수 있다!

## 심층 신경망으로 구현하기

In [38]:
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

In [39]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

In [40]:
model1 = tf.add(tf.matmul(L1, W2), b2) 

In [46]:
cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model1))
optimizer1 = tf.train.AdamOptimizer(learning_rate=0.01)
train_op1 = optimizer1.minimize(cost1)

optimizer들은 상황에 따라 좋은 함수가 달라 제공하는 다양한 ㄹ최적화 함수를 찾아보고 적용하는 것이 좋다.  
https://www.tensorflow.org/api_guides/python/train#Optimizers

In [48]:
# 텐서플로의 세션을 초기화한다.
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행한다.
    for step in range(100):
        sess.run(train_op1, feed_dict={X: x_data, Y: y_data})

        # 학습 도중 10번에 한 번씩 손실값을 출력해보자
        if (step + 1) % 10 == 0:
            print(step+1, sess.run(cost1, feed_dict={X: x_data, Y: y_data}))
    
    prediction = tf.argmax(model1, axis=1)
    target = tf.argmax(Y, axis=1)
    print('예측값: ', sess.run(prediction, feed_dict={X: x_data}))
    print('실제값: ', sess.run(target, feed_dict={Y: y_data}))
    
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

10 0.8315418
20 0.5876274
30 0.43293843
40 0.32342583
50 0.23843594
60 0.17338063
70 0.12656976
80 0.093792886
90 0.071269244
100 0.055701364
예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도: 100.00
